# 2025 세법 개정본 반영

In [6]:
pip install jpype1

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade langchain chromadb

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install requests-cache

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import re
from tqdm import tqdm
from tabula import read_pdf
from langchain.document_loaders import PyMuPDFLoader
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.cache import SQLiteCache
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from dotenv import load_dotenv

load_dotenv()

pdf_file = "data/tax_etc/2024_핵심_개정세법.pdf"

# SQLite 캐시 설정
SQLiteCache(".cache_prompt_etc.sqlite")

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 모든 PDF 처리
docs = []

# Tabula로 PDF에서 표 데이터 읽기
try:
    tables = read_pdf(pdf_file, pages="19-44,47-71,75-161", lattice=True, multiple_tables=True)
    table_texts = [table.to_string(index=False, header=True) for table in tables]
except Exception as e:
    print(f"Tabula Error with {pdf_file}: {e}")
    table_texts = []

# PyMuPDFLoader로 텍스트 데이터 로드 및 페이지 범위 필터링링
try:
    text_loader = PyMuPDFLoader(pdf_file)
    texts = text_loader.load()

    for i, text in enumerate(texts):
        text.metadata["page"] = i + 1      

    page_ranges = [(19, 44), (47, 71), (75, 161)]
    texts = [
        text for text in texts
        if any(start <= text.metadata.get("page", 0) <= end for start, end in page_ranges)
    ]
except Exception as e:
    print(f"PyMuPDFLoader Error with {pdf_file}: {e}")
    texts = []

# 표 텍스트를 Document 형식으로 변환
table_docs = [
    Document(page_content=text, metadata={"source": f"{pdf_file} - Table {i + 1}"})
    for i, text in enumerate(table_texts)
]

    # 텍스트를 Document 형식으로 변환
text_docs = [
    Document(page_content=text.page_content, metadata={"source": pdf_file})
    for text in texts
]

    # 텍스트 문서와 표 문서를 결합
docs.extend(text_docs + table_docs)

# 정규식 패턴화
update_docs = []

pattern1 = (
    r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
    r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
    r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
    r"\s*3\s*❚국민･기업\s*납세자용\s*|"
    r"\s*2\s*0\s*2\s*4\s|"
    r"\s한국세무사회\s|"    
)
pattern2 =r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])"


for doc in docs:
    if doc.page_content:
        edit_content = re.sub(pattern1, "", doc.page_content)
        edit_content = re.sub(pattern2, r"\1\2" , edit_content)
    else:
        edit_content = ""
    
    updated_docs = Document(page_content=edit_content, metadata=doc.metadata)   
    update_docs.append(updated_docs)

# 텍스트를 청크로 분리
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o",
    chunk_size=2000,
    chunk_overlap=100,
)

split_docs = splitter.split_documents(update_docs)

# Vector Store 생성
vector_store = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="tax_etc",
    persist_directory="vector_store/chroma/tax_etc_1"
)

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10}
)

c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 0 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 17 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3

In [2]:
update_docs[0]

Document(metadata={'source': 'data/tax_etc/2024_핵심_개정세법.pdf'}, page_content='2025 달라지는 세금제도\n \n01\n부동산 세금제도\n(소득법) 주택으로 용도변경 시 장기보유특별공제액 계산방법\n※ 2023년 개정사항\nㅇ건축물을 주택으로 용도 변경하여 1세대 1주택이 된 경우 각 용도기간별 보유･거주기간 공제율을 합산(①+ ②)하여 장기보유 특별공제액을 계산한다.\n①보유기간 공제율(최대 40%): 비주택 보유기간에 대한 일반 공제율+ 주택 보유기간에 대한 \n1세대 1주택 공제율\n②거주기간 공제율: 주택 거주기간에 대한 1세대 1주택 공제율\n<적용시기> ’25.1.1. 이후 양도하는 분부터 적용\n(소득법) 양도소득세 이월과세 적용대상 자산 확대\nㅇ증여를 통한 양도소득 세부담 회피 방지를 위해 이월과세 적용대상 자산에 양도일 전 1년 이내 증여받은 주식 등을 추가하였다.\n<적용시기> ’25.1.1. 이후 증여받는 분부터 적용\n(소득법) 토지･건물 일괄 취득･양도 시 안분계산 예외 신설\nㅇ토지와 건물 등을 함께 취득하거나 양도한 경우로서 그 토지와 건물 등을 구분 기장한 가액이 안분기준에 따라 안분계산한 가액과 30% 이상 차이가 있는 \n')

In [3]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [4]:
chain.invoke("친환경자동차의 개별소비세에 대해 변한 사항이 있어? ")

'네, 친환경자동차의 개별소비세에 관한 변경 사항이 있습니다. 2024년 개정세법에 따르면, 친환경자동차(하이브리드, 전기, 수소차)의 개별소비세 감면 적용기한이 2026년 12월 31일까지 2년 연장되었습니다. 또한, 하이브리드차의 경우 감면한도가 기존 100만원에서 70만원으로 축소되었습니다. 이 변경 사항은 2025년 1월 1일 이후에 반출하거나 수입신고하는 차량에 대해 적용됩니다.\n\n이 변경의 주된 이유는 친환경자동차 보급을 지원하기 위함입니다. 추가적인 질문이 있거나 더 궁금한 점이 있다면 언제든지 말씀해 주세요!'

In [4]:
chain.invoke("의료기술협력단을 기부금 손금산입 대상에 대해 변한 사항이 있어? ")


'의료기술협력단을 기부금 손금산입 대상에 포함하는 변경 사항이 있습니다. 법인세법에 따르면, 의료기술협력단은 특례기부금 단체와 일반기부금 단체에 추가되었습니다. 특례기부금 단체에 해당하는 병원이 설립하는 의료기술협력단은 2025년 1월 1일 이후 신고하는 분부터 적용되며, 일반기부금 단체에 해당하는 병원이 설립하는 의료기술협력단은 영 시행일 이후 신고하는 분부터 적용됩니다. 이 개정은 의료기술협력단 설립 및 운영을 지원하기 위한 것입니다.'

In [5]:
chain.invoke("연말정산시 필요한 서류")

'연말정산 시 필요한 서류는 다음과 같습니다:\n\n1. 소득·세액공제 증명서류: 연말정산간소화 서비스를 통해 출력하거나 전산파일로 내려받아야 합니다.\n2. 기부금명세서: 기부금을 공제받기 위해 필요합니다.\n3. 의료비지급명세서: 의료비 공제를 받기 위해 필요합니다.\n4. 신용카드 등 소득공제 신청서: 신용카드 사용에 대한 공제를 받기 위해 필요합니다.\n5. 월세액 세액공제 명세서: 월세에 대한 세액공제를 받기 위해 필요한 서류입니다.\n6. 기타 서류: 난임시술비나 취학 전아동 학원비 등 연말정산간소화 서비스에서 제공되지 않는 자료는 근로자가 직접 수집해야 합니다. \n\n각 서류는 회사에 제출하여 소득·세액공제신고서가 정확히 작성되었는지 확인받아야 합니다.'

In [9]:
chain.invoke("2023년 개정사항 중 2025년부터 시행한는 것들 알려줘")

'2023년 개정사항 중 2025년부터 시행되는 주요 내용은 다음과 같습니다:\n\n1. **소득세법 §95**: 건축물을 주택으로 용도 변경하여 1세대 1주택이 된 경우, 각 용도기간별 보유 및 거주기간 공제율을 합산하여 장기보유 특별공제액을 계산합니다. 이 변경 사항은 2025년 1월 1일 이후 양도하는 분부터 적용됩니다.\n\n2. **조세특례제한법 §91의19**: 장병내일준비적금 비과세 특례의 납입 한도가 월 40만원에서 월 55만원으로 상향됩니다. 이 변경은 2025년 1월 1일 이후 납입분부터 적용됩니다.\n\n3. **조세특례제한법 §105의3**: 개인택시 간이과세자가 구입하는 자동차에 대한 부가가치세 면세제도가 환급 제도로 전환됩니다. 이 변경은 2025년 1월 1일 이후 공급하는 분부터 적용됩니다.\n\n4. **소득세법 시행령 별표**: 소득파악 기반 확충을 위해 13개 업종이 현금영수증 의무발행 업종에 추가되고, 1개 업종이 정정됩니다. 이 변경은 2025년 1월 1일 이후 재화 또는 용역을 공급하는 분부터 적용됩니다.\n\n이 외에도 다양한 세법 개정사항이 2025년부터 시행될 예정입니다. 궁금한 사항이 있으면 구체적으로 질문해 주세요.'

In [7]:
chain.invoke("자료제공 동의를 신청한 부양가족의 소득 세액공제 일부 항목이 조회되지 않는 이유 알려줘")

'부양가족의 소득 세액공제 항목이 조회되지 않는 이유는 여러 가지가 있을 수 있습니다. 일반적으로 다음과 같은 이유로 인해 조회가 되지 않을 수 있습니다:\n\n1. **부양가족의 소득 기준 초과**: 부양가족의 소득이 일정 기준을 초과할 경우, 세액공제를 받을 수 없게 됩니다. 예를 들어, 부양가족의 연간 소득이 일정 금액(예: 100만원)을 초과하면 공제가 불가능할 수 있습니다.\n\n2. **자료제공 동의 미완료**: 부양가족이 세무서에 자료제공 동의를 완료하지 않았거나, 신청 과정에서 정보가 누락되었을 수 있습니다. 이 경우, 부양가족의 자료가 국세청에 제공되지 않아 조회되지 않을 수 있습니다.\n\n3. **해당 항목의 누락**: 부양가족의 소득 세액공제 항목이 세법 개정으로 인해 변경되었거나, 특정 항목이 공제 대상에서 제외되었을 수 있습니다.\n\n4. **정보 입력 오류**: 홈택스나 기타 세무 시스템에 부양가족의 정보를 잘못 입력했을 경우, 조회되지 않을 수 있습니다.\n\n이러한 경우, 먼저 자료제공 동의가 제대로 이루어졌는지 확인하고, 국세청에 문의하여 정확한 사유를 확인하는 것이 좋습니다. 추가 질문이나 구체적인 상황에 대해 더 알고 싶으시면, 구체적인 정보를 제공해 주세요.'

In [5]:
chain.invoke("연말정산이전 근로소득 원천징수 중점 확인사항 알려줘")

'연말정산 이전에 근로소득 원천징수와 관련하여 중점적으로 확인해야 할 사항은 다음과 같습니다:\n\n1. **소득 항목 확인**: 근로소득과 관련된 모든 소득 항목이 정확히 반영되었는지 확인해야 합니다. 특히, 상여금, 수당, 기타 보너스 등이 포함되어야 합니다.\n\n2. **원천징수세액 계산**: 소득세 및 지방소득세가 정확히 원천징수되었는지 확인해야 합니다. 이를 위해 소득금액에 맞는 세율이 적용되었는지 검토합니다.\n\n3. **공제 항목 반영**: 각종 공제 항목이 정확히 반영되었는지 확인합니다. 예를 들어, 인적공제, 특별공제, 연금저축공제 등이 올바르게 적용되었는지 확인합니다.\n\n4. **증빙서류 준비**: 연말정산 시 필요한 증빙서류(예: 기부금 영수증, 보험료 납입 증명서 등)를 미리 준비하여 누락되지 않도록 합니다.\n\n5. **기부금 영수증 발급**: 만약 기부를 하였다면, 전자기부금영수증 또는 종이 영수증이 정확히 발급되었는지 확인합니다. 2024년 개정 세법에 따르면 일정 금액 이상의 기부금에 대해서는 전자기부금영수증 발급이 의무화됩니다.\n\n6. **변경된 세법 확인**: 연말정산에 영향을 미칠 수 있는 최근 세법 개정사항을 확인하고, 변동사항이 있는 경우 이에 맞춰 준비해야 합니다.\n\n7. **근로자 정보 확인**: 본인의 인적사항(주소, 가족관계 등)이 정확히 반영되어 있는지 확인합니다. 잘못된 정보는 공제 항목에 영향을 줄 수 있습니다.\n\n위의 내용들은 연말정산 시 실수를 줄이고, 정확한 세액 계산을 위해 중요한 점검 사항입니다. 좀 더 구체적인 항목이 필요하거나 추가적인 질문이 있다면 알려주세요!'

In [6]:
chain.invoke("시골에 거주하는 부모님의 자료제공 동의는 어떻게 하는 것이 간편한가 알려줘")

'부모님의 자료제공 동의를 간편하게 받기 위해서는 다음과 같은 방법을 고려할 수 있습니다:\n\n1. **온라인 시스템 이용**: 국세청 홈택스와 같은 온라인 세금 시스템을 통해 자료제공 동의를 전자적으로 처리할 수 있습니다. 부모님께서 인터넷 사용에 익숙하시다면, 이 방법이 가장 빠르고 편리할 수 있습니다.\n\n2. **모바일 앱 활용**: 스마트폰을 사용하신다면, 국세청의 모바일 앱을 통해 자료제공 동의를 쉽게 진행할 수 있습니다. 앱을 다운로드하고 로그인한 후, 필요한 절차에 따라 동의를 완료할 수 있습니다.\n\n3. **공인인증서 또는 공동인증서 사용**: 온라인 서식 작성 시 공인인증서(공동인증서)를 사용하여 본인 인증을 하고 자료제공 동의를 진행할 수 있습니다. 부모님께서 이미 공인인증서를 가지고 계시다면, 이를 활용하면 간편합니다.\n\n4. **세무서 방문**: 만약 온라인 접근이 어려우시다면, 가까운 세무서를 방문하여 직원의 도움을 받아 자료제공 동의 절차를 밟을 수 있습니다. 이 경우 신분증과 필요한 서류를 지참해야 합니다.\n\n각 방법의 편리함은 부모님의 기술 숙련도와 접근 가능성에 따라 다를 수 있습니다. 온라인 방법이 가장 빠르고 간편하나, 필요 시 직접 방문하여 도움을 받는 것도 좋은 방법입니다.'